This script processes 3D image volumes in NRRD and TIFF formats by applying vesselness filtering, thresholding, and segmentation.

In [ ]:
import os
import slicer
import vtk, slicer
import VesselnessFiltering
from slicer.ScriptedLoadableModule import *

In [ ]:
def createEmptyVolume(imageSize, imageSpacing, nodeName):
    """
    Creates an empty volume in a 3D space with specified dimensions and spacing.

    Parameters:
    - imageSize (tuple of int): The dimensions of the image volume (x, y, z).
    - imageSpacing (tuple of float): The spacing between voxels in each dimension (x, y, z).
    - nodeName (str): The name to assign to the created volume node.

    Returns:
    - vtkMRMLScalarVolumeNode: The created volume node with the specified dimensions and spacing.
    """
    voxelType = vtk.VTK_FLOAT

    # Create an empty image volume
    imageData = vtk.vtkImageData()
    imageData.SetDimensions(imageSize)
    imageData.AllocateScalars(voxelType, 1)

    # Apply threshold to set all voxel values to 0
    thresholder = vtk.vtkImageThreshold()
    thresholder.SetInputData(imageData)
    thresholder.SetInValue(0)
    thresholder.SetOutValue(0)
    thresholder.Update()

    # Create volume node
    volumeNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLScalarVolumeNode", nodeName)
    volumeNode.SetSpacing(imageSpacing)
    volumeNode.SetAndObserveImageData(thresholder.GetOutput())
    volumeNode.CreateDefaultDisplayNodes()
    volumeNode.CreateDefaultStorageNode()

    return volumeNode  


def get_new_filename(filename):
    """
    Extracts the base name from a given filename, excluding the extension.

    Parameters:
    - filename (str): The full path or name of the file.

    Returns:
    - str: The base name of the file without the extension.
    """
    name, extension = os.path.splitext(filename)
    return name

In [ ]:
source_folder = ''
output_file_path = ''


for filename in os.listdir(source_folder):
    if filename.endswith('.tif') or filename.endswith('.nrrd'):
        print('Processing...', filename)
        input_volume_path = os.path.join(source_folder, filename)
        # load input volume
        _, InputVolumeNode = slicer.util.loadVolume(input_volume_path, returnNode=True)  # load input volume
        dim_in = InputVolumeNode.GetImageData().GetDimensions()  # save input volume shape
        spacing_in = InputVolumeNode.GetSpacing()  # save input volume spacing

        # Create output volume with same dim and spacing of InputVolume
        OutputNode = createEmptyVolume(dim_in, spacing_in, get_new_filename(filename) + '-VesselnessFiltered')  # invoke function
        # create vesselness filtering logic
        vfl = VesselnessFiltering.VesselnessFilteringLogic()  
        # compute VesselnessFiltering
        vfl.computeVesselnessVolume(InputVolumeNode, OutputNode,
                                    minimumDiameterMm=1,
                                    maximumDiameterMm=10,
                                    alpha=0.1,
                                    beta=0.9,
                                    contrastMeasure=200)

        slicer.util.saveNode(OutputNode, os.path.join(output_file_path, get_new_filename(filename) + '-VesselnessFiltered.nrrd'))
        labelVolumeNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLLabelMapVolumeNode")
        slicer.vtkSlicerVolumesLogic().CreateLabelVolumeFromVolume(slicer.mrmlScene, labelVolumeNode, OutputNode)
        thresholdValue = 0.2  # set a threshold value
        voxelArray = slicer.util.arrayFromVolume(OutputNode)  # extract numpy array from output volume
        labelVoxelArray = slicer.util.arrayFromVolume(labelVolumeNode)  # extract numpy array from labelmap volume
        labelVoxelArray[voxelArray < thresholdValue] = 0 # set all voxels below thr to 0
        labelVoxelArray[voxelArray >= thresholdValue] = 1 # set all voxels above thr to 1
        segmentationNode = slicer.mrmlScene.AddNewNodeByClass('vtkMRMLSegmentationNode')  # create segmentation node
        slicer.modules.segmentations.logic().ImportLabelmapToSegmentationNode(labelVolumeNode, segmentationNode)
#         segmentationNode.CreateClosedSurfaceRepresentation()  # make segmentation visible in 3D
        reference = slicer.util.getNode(get_new_filename(filename) + '-VesselnessFiltered') # this will be the volume the segmentation was drawn on
        labelmapVolumeNodeClosed = slicer.mrmlScene.AddNewNodeByClass('vtkMRMLLabelMapVolumeNode')
        seg = slicer.util.getNode("Segmentation")  # set segmentation node            
        ids = vtk.vtkStringArray()
        seg.GetDisplayNode().GetVisibleSegmentIDs(ids)
        smsl = slicer.modules.segmentations.logic()
        smsl.ExportSegmentsToLabelmapNode(segmentationNode,ids,labelmapVolumeNodeClosed,reference)

        dim_out_lab = labelmapVolumeNodeClosed.GetImageData().GetDimensions()  # save output shape
        spacing_out_lab = labelmapVolumeNodeClosed.GetSpacing()  # save output spacing
        print(get_new_filename(filename))
        slicer.util.saveNode(labelmapVolumeNodeClosed, os.path.join(output_file_path, get_new_filename(filename) + '-segment.tif'))


        # Close all open volumes
        # slicer.mrmlScene.Clear()